# Where is the safest county and state to live in the US?  By Jake Lieberfarb

The United States is large country and has all forms of topography. From the rain forests of Washington State to the marshes of the Everglades in Florda, these beautiful areas of the USA offer different locations to live. However, increase in heat in the summer months, tornadoes and wildfires take the lives of hundreds of people each year. To avoid many of these natural disaster the question is present of "where is the safest place to live in the US?" To address this question, the [NOAA dataset](https://www.ncdc.noaa.gov/stormevents/ftp.jsp) on all recorded storm events from 2018 and 2019. For this analysis "safe" was defined as the state and county that has the lease number of storm events and has the relatively lowest property damage per the size of the state. Once a state was selected, the states counties were analyzed to see which state and respective county would have the least number of storm events between 2018 and 2019.

In [282]:
#import necessary modules 
import pandas as pd
from IPython.display import display
import plotly
import chart_studio.plotly as py
import plotly.graph_objects as go

In [283]:
#read in the data
pd.set_option('display.precision', 10)
pd.set_option('display.max_columns', 100)
#my plotly access codes
py.sign_in('Jlieberfarb', 'vBRbr33wqGdlxzikJ0Qx')
d_18_str = pd.read_csv("StormEvents_details-ftp_v1.0_d2018_c20200819.csv",index_col=0)
d_19_str = pd.read_csv("StormEvents_details-ftp_v1.0_d2019_c20201017.csv",index_col=0)
#fill in 0 for NaN
d_18=d_18_str.fillna(0)
d_19=d_19_str.fillna(0)


In [284]:
#clean 2018 data 
#Searches the column and creates a list of trues and falses and removes the falses
#remove american territorites, dc, and bodies of water
d_18= d_18[~d_18['STATE'].isin(['DISTRICT OF COLUMBIA','E PACIFIC', 'AMERICAN SOMOA', 'GUAM', 'VIRGIN ISLANDS', 'PUERTO RICO', 'LAKE HURON', 
                                'ATLANTIC NORTH', 'ATLANTIC SOUTH', 'GULF OF MEXICO','LAKE MICHIGAN', 'LAKE ERIE', 
                                'LAKE SUPERIOR', 'HAWAII WATERS', 'LAKE ST CLAIR', 'LAKE ONTARIO', 'ST LAWRENCE R'])]

#lays out all abbreviations that will appear in the column
mapping = dict(K='E3', 
               M='E6',
               B='E9') #everything that matches these keys (K,M,B) is now mapped to the value

DAMAGE_PROPERTY= d_18['DAMAGE_PROPERTY']
#removes abbreviation and the regex statement fills in the letters for their corresponding values from mapping
#assign function used to create new dataframe
d_18= d_18.assign(DAMAGE_PROPERTY=pd.to_numeric(DAMAGE_PROPERTY.replace(mapping, regex=True))) 
#d_18.DAMAGE_PROPERTY.astype("float64")

In [285]:
#clean 2019 data
d_19= d_19[~d_19['STATE'].isin(['DISTRICT OF COLUMBIA','E PACIFIC', 'AMERICAN SAMOA', 'GUAM', 'VIRGIN ISLANDS', 'PUERTO RICO', 'LAKE HURON', 
                                'ATLANTIC NORTH', 'ATLANTIC SOUTH', 'GULF OF MEXICO','LAKE MICHIGAN', 'LAKE ERIE', 
                                'LAKE SUPERIOR', 'HAWAII WATERS', 'LAKE ST CLAIR', 'LAKE ONTARIO', 'ST LAWRENCE R'])]

mapping = dict(K='E3', 
               M='E6',
               B='E9') 

DAMAGE_PROPERTY= d_19['DAMAGE_PROPERTY']

d_19= d_19.assign(DAMAGE_PROPERTY=pd.to_numeric(DAMAGE_PROPERTY.replace(mapping, regex=True))) 

# Which storm event causes the most property damage by state? 

Choropleth, histograms, and bar charts were constructed to answer this question. The total property damage for each state and the total damage caused by each storm event were aggregated and used in this analysis. Also, the proportion of property damage to the states' size were taken into account as well. The choropleth was used to get a hollistic understanding of the data and see if there were any collections of states in a certain areas of the country that had high or low property damage. The histogram was introduced to get an understanding of the spread of the data and see if there were any outliers. The bar charts were utilized to breakdown why certain states would be having such a high or low total property damage and if there are certain storm events that cause them. 

In [286]:
#Overview of top 10 and bottom 10 states by Total damage
td= pd.DataFrame(d_18, columns = ['STATE','DAMAGE_PROPERTY'])
tdd= td.groupby(["STATE"]).DAMAGE_PROPERTY.sum().reset_index()
tdd.sort_values( by= 'DAMAGE_PROPERTY', ascending = False, inplace = True)
tdd.head(10)

,STATE,DAMAGE_PROPERTY
5,CALIFORNIA,19233667900.0
9,FLORIDA,4209301160.0
33,NORTH CAROLINA,2023526460.0
10,GEORGIA,1545984010.0
6,COLORADO,669220000.0
15,IOWA,346347000.0
43,TEXAS,338081250.0
22,MICHIGAN,201517500.0
49,WISCONSIN,181952660.0
13,ILLINOIS,143710000.0


In [287]:
tdd.tail(10)

,STATE,DAMAGE_PROPERTY
40,SOUTH CAROLINA,5914550.0
41,SOUTH DAKOTA,5179000.0
45,VERMONT,5054000.0
36,OKLAHOMA,4675000.0
30,NEW JERSEY,4142000.0
1,ALASKA,3589000.0
29,NEW HAMPSHIRE,3445000.0
37,OREGON,1231200.0
39,RHODE ISLAND,1153000.0
8,DELAWARE,0.0


In [288]:
#add state area into calculating accurate damage 
#pulled data from https://statesymbolsusa.org/symbol-official-item/national-us/uncategorized/states-size
state_size = {
    'Pennsylvania': 44820, 'Florida': 53997, 'Alaska': 570641, 'Kentucky': 39732, 
    'Hawaii': 6423, 'Nebraska': 76878, 'Missouri': 68898, 'Ohio': 40953, 
    'Alabama': 50750, 'Rhode Island': 1034, 'South Dakota': 75898, 
    'Colorado': 103730, 'New Jersey': 7419, 'Washington': 66582, 
    'North Carolina': 48718, 'New York': 47224, 'Texas': 261914,   
    'Mississippi': 46914, 'Oklahoma': 68679, 'Delaware': 1955, 'Minnesota': 79617, 
    'Illinois': 55593, 'Arkansas': 52075, 'Nevada': 109806, 'Maine': 30865,
    'New Mexico': 121365, 'Indiana': 35870, 'Maryland': 9775, 'Louisiana': 43566, 
    'Idaho': 82751, 'Wyoming': 97105, 'Tennessee': 41220, 'Arizona': 113642, 
    'Iowa': 55875, 'Michigan': 56539, 'Kansas': 81823, 'Utah': 82168, 
    'Virginia': 39598, 'Oregon': 96003, 'Connecticut': 4845, 'Montana': 145556, 
    'California': 155973, 'Massachusetts': 7838, 'West Virginia': 24087, 
    'South Carolina': 30111, 'New Hampshire': 8969, 'Wisconsin': 54314,
    'Vermont': 9249, 'Georgia': 57919, 'North Dakota': 68994, 
    }
#separating the state from their size then capitalizing the state names and putting it back 
#together with its corresponding size
# creating a list of all the keys in the above dictionary which are the same names as STATE
statek= state_size.keys()
#changes all the states to upper case so it matches the tdd dataframe
statek= [state.upper()for state in statek]
#creating a new list of the state sizes
state_values=state_size.values()
#zip combines state name and their abbreviation and dict turns it into a dictionary
state_size=dict(zip(statek,state_values))
# create new column to be added to tdd which will place the STATE CODES next to their state
tdd['STATE_SIZE'] = tdd['STATE'].map(state_size)
#take into account the size of a state and its damages
tdd['PROP']= tdd['DAMAGE_PROPERTY']/tdd['STATE_SIZE']
tdd.sort_values( by= 'PROP', ascending = False, inplace = True)
tdd.tail(10)

,STATE,DAMAGE_PROPERTY,STATE_SIZE,PROP
25,MISSOURI,9264000.0,68898.0,134.4596359836
50,WYOMING,10196000.0,97105.0,104.9997425467
26,MONTANA,10593000.0,145556.0,72.7761136607
41,SOUTH DAKOTA,5179000.0,75898.0,68.2363171625
36,OKLAHOMA,4675000.0,68679.0,68.0702980533
31,NEW MEXICO,8105000.0,121365.0,66.7820211758
37,OREGON,1231200.0,96003.0,12.8245992313
1,ALASKA,3589000.0,570641.0,6.2894183909
8,DELAWARE,0.0,1955.0,0.0000000000
2,AMERICAN SAMOA,50137000.0,NaN,NaN


In [289]:
#plotly only plots state codes so need to add column of state codes to dataframe
state_codes = {
    'Pennsylvania': 'PA', 'Florida': 'FL', 'Alaska': 'AK', 'Kentucky': 'KY', 
    'Hawaii': 'HI', 'Nebraska': 'NE', 'Missouri': 'MO', 'Ohio': 'OH', 
    'Alabama': 'AL', 'Rhode Island': 'RI', 'South Dakota': 'SD', 
    'Colorado': 'CO', 'New Jersey': 'NJ', 'Washington': 'WA', 
    'North Carolina': 'NC', 'New York': 'NY', 'Texas': 'TX',   
    'District of Columbia' : 'DC','Mississippi': 'MS', 'Oklahoma': 'OK', 
    'Delaware': 'DE', 'Minnesota': 'MN', 'Illinois': 'IL', 'Arkansas': 'AR', 
    'New Mexico': 'NM', 'Indiana': 'IN', 'Maryland': 'MD', 'Louisiana': 'LA', 
    'Idaho': 'ID', 'Wyoming': 'WY', 'Tennessee': 'TN', 'Arizona': 'AZ', 
    'Iowa': 'IA', 'Michigan': 'MI', 'Kansas': 'KS', 'Utah': 'UT', 
    'Virginia': 'VA', 'Oregon': 'OR', 'Connecticut': 'CT', 'Montana': 'MT', 
    'California': 'CA', 'Massachusetts': 'MA', 'West Virginia': 'WV', 
    'South Carolina': 'SC', 'New Hampshire': 'NH', 'Wisconsin': 'WI',
    'Vermont': 'VT', 'Georgia': 'GA', 'North Dakota': 'ND', 
    'Nevada': 'NV', 'Maine': 'ME'}

statekeys= state_codes.keys()
statekeys= [state.upper()for state in statekeys]
code_values=state_codes.values()
state_codes=dict(zip(statekeys,code_values))
tdd['STATE_CODE'] = tdd['STATE'].map(state_codes)

from plotly.subplots import make_subplots
# had originally tried to make subplot with both choropleth maps but
#could not get side bar to separate for each map
#spec = layout format 
fig = make_subplots(rows=1,cols=1, specs=[[{"type": "geo"}]])
fig.add_trace(go.Choropleth(
    locations=tdd['STATE_CODE'],
    z = tdd['DAMAGE_PROPERTY'].astype(float), 
    locationmode = 'USA-states',
    text=tdd['STATE'],
    colorscale = 'Purples',
    hoverinfo='location+z',
    colorbar_title = "Property Damage"), row=1, col =1)

fig.update_layout( height = 540,
        geo_scope='usa',
        title_text= '2018 Property Damage by state with California')

In [290]:
#remove CALIFORNIA
fig = make_subplots(rows=1,cols=1, specs=[[{"type": "geo"}]])

tdd_noCal= tdd[tdd['STATE']!= 'CALIFORNIA']
fig.add_trace(go.Choropleth(
    locations=tdd_noCal['STATE_CODE'], # Spatial coordinates
    z = tdd_noCal['DAMAGE_PROPERTY'].astype(float), 
    locationmode = 'USA-states',
    text=tdd_noCal['STATE'],
    colorscale = 'Purples',
    hoverinfo='location+z',
    colorbar_title = "Property Damage"),row=1, col =1)

fig.update_layout( height = 540,
        geo_scope='usa',
        title_text = '2018 Property Damage by state without California')

In [291]:
#normalize data for better understanding of spread
mean_dp = tdd.DAMAGE_PROPERTY.mean()
max_dp = tdd.DAMAGE_PROPERTY.max()
min_dp = tdd.DAMAGE_PROPERTY.min()

tdd['DAMAGE_PROPERTY'] = tdd['DAMAGE_PROPERTY'].apply(lambda x: (x - mean_dp ) / (max_dp -min_dp ))
fig = go.Figure(data=[go.Histogram(x=tdd.DAMAGE_PROPERTY)])
fig.update_layout( height = 540,
        title_text = 'Histogram of 2018 Property Damage'
        )
fig.show()

In [292]:
#2019 data on property damage
td_19= pd.DataFrame(d_19, columns = ['STATE','DAMAGE_PROPERTY'])
tdd_19= td_19.groupby(["STATE"]).DAMAGE_PROPERTY.sum().reset_index()
tdd_19.sort_values( by= 'DAMAGE_PROPERTY', ascending = False, inplace = True)
tdd_19.head(10)

,STATE,DAMAGE_PROPERTY
42,TEXAS,3092746900.0
34,OHIO,691652900.0
26,NEBRASKA,641041500.0
24,MISSOURI,222476500.0
21,MICHIGAN,203160500.0
17,LOUISIANA,186733000.0
32,NORTH CAROLINA,149250600.0
4,CALIFORNIA,144124900.0
40,SOUTH DAKOTA,124524950.0
41,TENNESSEE,122670330.0


In [293]:
tdd_19.tail(10)

,STATE,DAMAGE_PROPERTY
49,WYOMING,1329500.0
0,ALABAMA,981600.0
29,NEW JERSEY,934000.0
6,CONNECTICUT,803000.0
27,NEVADA,438000.0
18,MAINE,420000.0
43,UTAH,293000.0
10,HAWAII,254250.0
38,RHODE ISLAND,142700.0
7,DELAWARE,0.0


In [294]:
state_size = {
    'Pennsylvania': 44820, 'Florida': 53997, 'Alaska': 570641, 'Kentucky': 39732, 
    'Hawaii': 6423, 'Nebraska': 76878, 'Missouri': 68898, 'Ohio': 40953, 
    'Alabama': 50750, 'Rhode Island': 1034, 'South Dakota': 75898, 
    'Colorado': 103730, 'New Jersey': 7419, 'Washington': 66582, 
    'North Carolina': 48718, 'New York': 47224, 'Texas': 261914,   
    'Mississippi': 46914, 'Oklahoma': 68679, 'Delaware': 1955, 'Minnesota': 79617, 
    'Illinois': 55593, 'Arkansas': 52075, 'Nevada': 109806, 'Maine': 30865,
    'New Mexico': 121365, 'Indiana': 35870, 'Maryland': 9775, 'Louisiana': 43566, 
    'Idaho': 82751, 'Wyoming': 97105, 'Tennessee': 41220, 'Arizona': 113642, 
    'Iowa': 55875, 'Michigan': 56539, 'Kansas': 81823, 'Utah': 82168, 
    'Virginia': 39598, 'Oregon': 96003, 'Connecticut': 4845, 'Montana': 145556, 
    'California': 155973, 'Massachusetts': 7838, 'West Virginia': 24087, 
    'South Carolina': 30111, 'New Hampshire': 8969, 'Wisconsin': 54314,
    'Vermont': 9249, 'Georgia': 57919, 'North Dakota': 68994, 
    }

statek= state_size.keys()

statek= [state.upper()for state in statek]

state_values=state_size.values()

state_size=dict(zip(statek,state_values))

tdd_19['STATE_SIZE'] = tdd_19['STATE'].map(state_size)

tdd_19['PROP']= tdd_19['DAMAGE_PROPERTY']/tdd_19['STATE_SIZE']
tdd_19.sort_values( by= 'PROP', ascending = False, inplace = True)
tdd_19.tail(10)

,STATE,DAMAGE_PROPERTY,STATE_SIZE,PROP
10,HAWAII,254250.0,6423,39.5843063989
25,MONTANA,3960000.0,145556,27.2060237984
0,ALABAMA,981600.0,50750,19.3418719212
1,ALASKA,8545000.0,570641,14.9743884509
5,COLORADO,1420800.0,103730,13.6970982358
49,WYOMING,1329500.0,97105,13.6913650172
18,MAINE,420000.0,30865,13.6076462012
27,NEVADA,438000.0,109806,3.9888530681
43,UTAH,293000.0,82168,3.5658650570
7,DELAWARE,0.0,1955,0.0000000000


In [295]:
#plotly only plots state codes so need to add column of state codes to dataframe
state_codes = {
    'Texas': 'TX','California': 'CA', 'Massachusetts': 'MA', 'West Virginia': 'WV', 
    'South Carolina': 'SC', 'New Hampshire': 'NH', 'Wisconsin': 'WI',
    'Vermont': 'VT', 'Georgia': 'GA', 'North Dakota': 'ND', 
    'Pennsylvania': 'PA', 'Florida': 'FL', 'Alaska': 'AK', 'Kentucky': 'KY', 
    'Hawaii': 'HI', 'Nebraska': 'NE', 'Missouri': 'MO', 'Ohio': 'OH', 
    'Alabama': 'AL', 'Rhode Island': 'RI', 'South Dakota': 'SD', 
    'Colorado': 'CO', 'New Jersey': 'NJ', 'Washington': 'WA', 
    'North Carolina': 'NC', 'New York': 'NY', 
    'Nevada': 'NV','District of Columbia' : 'DC','Mississippi': 'MS', 'Oklahoma': 'OK', 
    'Delaware': 'DE', 'Minnesota': 'MN', 'Illinois': 'IL', 'Arkansas': 'AR', 
    'New Mexico': 'NM', 'Indiana': 'IN', 'Maryland': 'MD', 'Louisiana': 'LA', 
    'Idaho': 'ID', 'Wyoming': 'WY', 'Tennessee': 'TN', 'Arizona': 'AZ', 
    'Iowa': 'IA', 'Michigan': 'MI', 'Kansas': 'KS', 'Utah': 'UT', 
    'Virginia': 'VA', 'Oregon': 'OR', 'Connecticut': 'CT', 'Montana': 'MT', 
     'Maine': 'ME'}

statekeys= state_codes.keys()

statekeys= [state.upper()for state in statekeys]

code_values=state_codes.values()

state_codes=dict(zip(statekeys,code_values))

tdd_19['STATE_CODE'] = tdd_19['STATE'].map(state_codes)

fig = make_subplots(rows=1,cols=1, specs=[[{"type": "geo"}]])
fig.add_trace(go.Choropleth(
    locations=tdd_19['STATE_CODE'],
    z = tdd_19['DAMAGE_PROPERTY'].astype(float), 
    locationmode = 'USA-states',
    text=tdd_19['STATE'],
    colorscale = 'Oranges',
    hoverinfo='location+z',
    colorbar_title = "Property Damage"), row=1, col =1)

fig.update_layout( height = 540,
        geo_scope='usa',
        title_text= '2019 Property Damage by state with Texas')

In [296]:
#remove CALIFORNIA
fig = make_subplots(rows=1,cols=1, specs=[[{"type": "geo"}]])

tdd_noTx= tdd_19[tdd_19['STATE']!= 'TEXAS']
fig.add_trace(go.Choropleth(
    locations=tdd_noTx['STATE_CODE'], # Spatial coordinates
    z = tdd_noTx['DAMAGE_PROPERTY'].astype(float), 
    locationmode = 'USA-states',
    text=tdd_noTx['STATE'],
    colorscale = 'Oranges',
    hoverinfo='location+z',
    colorbar_title = "Property Damage"),row=1, col =1)

fig.update_layout( height = 540,
        geo_scope='usa',
        title_text = '2019 Property Damage by state without Texas')

In [297]:
#normalize data for better understanding of spread
mean_dp = tdd_19.DAMAGE_PROPERTY.mean()
max_dp = tdd_19.DAMAGE_PROPERTY.max()
min_dp = tdd_19.DAMAGE_PROPERTY.min()

tdd_19['DAMAGE_PROPERTY'] = tdd_19['DAMAGE_PROPERTY'].apply(lambda x: (x - mean_dp ) / (max_dp -min_dp ))

fig = go.Figure(data=[go.Histogram(x=tdd_19.DAMAGE_PROPERTY)])

fig.update_layout( height = 540,
        title_text = 'Histogram of 2019 Property Damage'
        )

fig.show()

In [298]:
#2018 property damage by state 
bar_data= d_18.groupby(['STATE','EVENT_TYPE'], 
                       as_index=False).agg({'DAMAGE_PROPERTY': 'sum'})

states = list(bar_data['STATE'].unique()) 

fig = go.Figure()
#here there is a loop through all the states from the list above and on the x bar it lists the types of 
#weather events in descending order by the total amount of property damage each weather event causes.  
#it will then show the event type on the x axis.
#the loop iterates through the list of states until it matches the state it is looking for. 
#the y bar sorts the types of weather events by the total amount of property damage each event causes and 
#sorts them from greatest to least. 
for state in states:
    fig.add_trace(
                go.Bar(x=bar_data[bar_data['STATE'] == state].sort_values(
                        by= 'DAMAGE_PROPERTY', ascending = False)['EVENT_TYPE'], 
                       y= bar_data[bar_data['STATE'] == state].sort_values(
                        by= 'DAMAGE_PROPERTY', ascending = False)['DAMAGE_PROPERTY'], 
                       visible= False))

buttons= []

# when going through the states in the drop down, only allows you to select one state at a time
# this loops through the indexes of the list of states and it allows us
# to select specific items based on their index to update the visible list
for i in range(len(states)):
    visible = [False]*len(states)
    if i == 0: 
        visible[i]= True
    else:
        visible[i] = True
        visible[i-1] = False
#creating the dropdown menu     
    buttons.append(dict(label = states[i],
                        method = 'update',           
                        args = [{'visible': visible}]))
#updating the menu so it has the buttoms      
fig.update_layout(updatemenus=list([
    dict(buttons = buttons)]))

fig.update_layout(title = '2018 Property Damage by State ',
                 height = 540)
fig.show()

In [299]:
#2019 property damage by state 
bar_data= d_19.groupby(['STATE','EVENT_TYPE'], 
                       as_index=False).agg({'DAMAGE_PROPERTY': 'sum'})

states = list(bar_data['STATE'].unique()) 

fig = go.Figure()

for state in states:
    fig.add_trace(
                go.Bar(x=bar_data[bar_data['STATE'] == state].sort_values(
                       by= 'DAMAGE_PROPERTY', ascending = False)['EVENT_TYPE'], 
                       y= bar_data[bar_data['STATE'] == state].sort_values(
                       by= 'DAMAGE_PROPERTY', ascending = False)['DAMAGE_PROPERTY'], 
                       visible= False))

buttons= []

for i in range(len(states)):
    visible = [False]*len(states)
    if i == 0: 
        visible[i]= True
    else:
        visible[i] = True
        visible[i-1] = False
 
    buttons.append(dict(label = states[i],
                        method = 'update',           
                        args = [{'visible': visible}]))
 
fig.update_layout(updatemenus=list([
    dict(buttons = buttons)]))

fig.update_layout(title = '2019 Property Damage by State ',
                 height = 540)
fig.show()

The chloropleth map  for 2018 revealed that the highest property damage were for California, Florida, North Carolina, Georgia, and Colorado. California was much higher at around nineteen billion dollars while Florida came in second at around four billion dollars. The states with the lowest property damage for 2018 were  Rhode Island, Oregon and New Hampshire. Rhode Island had around one million dollars in damages and New Hampshire had around three and a half million dollars in damages.

When taking state size into account, Delaware was still the lowest as it had no property damage for both 2018 and 2019. The next state to be consistently in the lower 10 of states with smallest property damage per state size was Wyoming. In 2018, Wyoming had wildfires that caused 10 million dollars in damages and in 2019 it was hail at one million that caused most of the damages. 
For 2019 the top five stats for property damage are Texas at three billion, Ohio almost 700 million, Nebraska at around 650 million, and Missouri at 200 million. Rhode Island was the only state in the lower range of property damage at 142 thousand dollars. Bar charts were brought in to what may be causing such states to have high property damage.  For California, most of their Property Damage came from wildfires at about 18.5 billion. Debris Flow came in second at around 600 million. Florida had hurricane damage costing about three billion.  Most of Rhode Island’s property damage was brought on by a Tornado which did one million dollars in damages. In 2019 Flash Floods did a little over 100 million dollars in damages and wildfires did about 13 million in damages. Tornadoes caused two billion dollars in damages for Texas. Interestingly, Delaware did not have any property damage for 2018 and 2019. As we will see in the next section, this does not mean Delaware did not have any storm events, it just means that there they did not cause any property damage. 

In 2018 it was wildfires and hurricanes that caused the most damage by state for California and Florida, respectively. In 2019 Tornadoes caused the most damages for Texas and Strong Winds for Rhode Island. The big outlier here is Delaware as it did nto have any property damage for both years. When taking the proprotion of state size to property damage, Wyoming is consistently in the lower range.  

# What are the most common Storm Events?  

The frequency of each of the storm events were investigated individually (i.e. how many floods in 2018) and by state. This was done by building a box plot to see the top ten occurring storm events by year and a pie chart to see if the selected states had a high frequency of any of these events. 

In [300]:
#overview of 2018 storm events

#aggregates number of events into each state and counting them. 
#include EPISODE_ID as no null vales (wont count if null)
#pie_data will be used later for pie charts
pie_data= d_18.groupby(['STATE','EVENT_TYPE'], as_index=False).agg({'EPISODE_ID': 'count'})

overview = pie_data.groupby('EVENT_TYPE', as_index= False).sum()
overview.sort_values('EPISODE_ID', ascending = False, inplace=True)

fig = go.Figure()
fig.add_trace( go.Bar(x= overview['EVENT_TYPE'].head(10), y= overview['EPISODE_ID']))

fig.update_layout(title = '2018 most common storm events',
                 height = 540)

In [301]:
#2018 Storm Events

#Only want states to appear once. converting to a list makes collection 1-d and easier to iterate through
states = list(pie_data['STATE'].unique()) 

fig = go.Figure()

for state in states:
    fig.add_trace(
                go.Pie(labels=pie_data[pie_data['STATE'] == state]['EVENT_TYPE'], 
                       values= pie_data[pie_data['STATE'] == state]['EPISODE_ID'], 
                       hoverinfo='label+percent+value',
                       textinfo= 'none',
                       visible= False))

buttons= []


for i in range(len(states)):
    visible = [False]*len(states)
    if i == 0: 
        visible[i]= True
    else:
        visible[i] = True
        visible[i-1] = False

    buttons.append(dict(label = states[i],
                        method = 'update',           
                        args = [{'visible': visible},               
                                {'title': '2018 Storm Events'}]))
  
fig.update_layout(updatemenus=list([
    dict(buttons = buttons)]))

fig.update_layout(title = '2018 Storm Events',
                 height = 600)
fig.show()

In [302]:
#overview of 2019 storm events
pie_data_19= d_19.groupby(['STATE','EVENT_TYPE'], as_index=False).agg({'EPISODE_ID': 'count'})

overview = pie_data_19.groupby('EVENT_TYPE', as_index= False).sum()
overview.sort_values('EPISODE_ID', ascending = False, inplace=True)

fig = go.Figure()
fig.add_trace( go.Bar(x= overview['EVENT_TYPE'].head(10), y= overview['EPISODE_ID']))

fig.update_layout(title = '2019 Most Common Storm Events',
                 height = 540)

In [303]:
#2019 Storm Events
states = list(pie_data_19['STATE'].unique()) 

fig = go.Figure()

for state in states:
    fig.add_trace(
                go.Pie(labels=pie_data_19[pie_data_19['STATE'] == state]['EVENT_TYPE'], 
                       values= pie_data_19[pie_data_19['STATE'] == state]['EPISODE_ID'], 
                       hoverinfo='label+percent+value',
                       textinfo= 'none',
                       visible= False))

buttons= []

for i in range(len(states)):
    visible = [False]*len(states)
    if i == 0: 
        visible[i]= True
    else:
        visible[i] = True
        visible[i-1] = False
  
    buttons.append(dict(label = states[i],
                        method = 'update',           
                        args = [{'visible': visible}]))

fig.update_layout(updatemenus=list([
    dict(buttons = buttons)]))

fig.update_layout(title = '2019 Storm Events',
                 height = 600)
fig.show()

The most common storm events in 2018 and 2019 were thunderstorms, hail, and floods. However, 2019 saw more storm events than 2018.  Specifically,  2018 had 14 thousand thunderstorms and 2019 had around 18 thousand thunderstorms. The pie charts were useful in breaking down the frequency of storm events by state. In particular states such as California ’s most common storm event was high winds(228) and only had 40 wild fires but those wildfires caused most of the property damage for the state. Delaware had  19 flash floods as its most common storm event in 2018 and had 6 flash floods in 2019.  

In 2018, Wyoming had high winds(564), and hail(259) as the most common storm event. In 2019 it was hail(319),  high winds (236), and only four wildfires. When considering moving to a location, it is important to have an understanding of what could be the potential storm events one would encounter.This section was useful in understanding the different types of events that affect the states of Rhode Island, Wyoming, and California. Each year is different in the frequency of these states and the storm events that affect them.  

# What are the causes of deaths by state and event?

An analysis was conducted to see what were the most common causes for deaths from storm events by state and then which storm events causes the most in each state. This was done by constructing four bar charts and searching through by state what their storm event deaths are and if other states around them had anything similar. As prior analysis has shown, weather is very unpredictable and by looking that the different types of death by state, one could gage if an area is safe if they are trying to avoid excessive heat, hurricanes, etc. 

In [304]:
tot_18_d= pd.DataFrame(d_18, columns = ['STATE','EVENT_TYPE','INJURIES_DIRECT','DEATHS_DIRECT'])
tot_18_dd= tot_18_d.groupby(["STATE", 'EVENT_TYPE'], as_index=False).sum()
tot_18_dd.sort_values( by= 'DEATHS_DIRECT', ascending = False, inplace = True)
tot_18_dd.rename({'DEATHS_DIRECT': 'DEATHS_DIRECT_2018', 'INJURIES_DIRECT': 'INJURIES_DIRECT_2018'}, axis=1, inplace=True)
tot_18_dd

events = list(tot_18_dd['EVENT_TYPE'].unique()) 

fig = go.Figure()

for event in events:
    fig.add_trace(
                go.Bar(x=tot_18_dd[tot_18_dd['EVENT_TYPE'] == event].sort_values(
                    by= 'DEATHS_DIRECT_2018',ascending = True)['STATE'].tail(10), 
                       y= tot_18_dd[tot_18_dd['EVENT_TYPE'] == event].sort_values(
                    by= 'DEATHS_DIRECT_2018',ascending = True)['DEATHS_DIRECT_2018'].tail(10), 
                       visible= False))

buttons= []

for i in range(len(events)):
    visible = [False]*len(events)
    if i == 0: 
        visible[i]= True
    else:
        visible[i] = True
        visible[i-1] = False
    
    buttons.append(dict(label = events[i],
                        method = 'update',           
                        args = [{'visible': visible}]))
fig.update_layout(updatemenus=list([
    dict(buttons = buttons)]))

fig.update_layout(title = 'Top 10 Fatalities of States by Event for 2018',
                 height = 540)
fig.show()

In [305]:
tot_19_d= pd.DataFrame(d_19, columns = ['STATE','EVENT_TYPE','INJURIES_DIRECT','DEATHS_DIRECT'])
tot_19_dd= tot_19_d.groupby(["STATE", 'EVENT_TYPE'], as_index=False).sum()
tot_19_dd.sort_values( by= 'DEATHS_DIRECT', ascending = False, inplace = True)
tot_19_dd.rename({'DEATHS_DIRECT': 'DEATHS_DIRECT_2019', 'INJURIES_DIRECT': 'INJURIES_DIRECT_2019'}, axis=1, 
                 inplace=True)
tot_19_dd

events = list(tot_19_dd['EVENT_TYPE'].unique()) 

fig = go.Figure()

for event in events:
    fig.add_trace(
                go.Bar(x=tot_19_dd[tot_19_dd['EVENT_TYPE'] == event].sort_values(
                    by= 'DEATHS_DIRECT_2019',ascending = True)['STATE'].tail(10), 
                       y= tot_19_dd[tot_19_dd['EVENT_TYPE'] == event].sort_values(
                    by= 'DEATHS_DIRECT_2019',ascending = True)['DEATHS_DIRECT_2019'].tail(10), 
                       visible= False))

buttons= []


for i in range(len(events)):
    visible = [False]*len(events)
    if i == 0: 
        visible[i]= True
    else:
        visible[i] = True
        visible[i-1] = False
  
    buttons.append(dict(label = events[i],
                        method = 'update',           
                        args = [{'visible': visible}]))
   
fig.update_layout(updatemenus=list([
    dict(buttons = buttons)]))

fig.update_layout(title = 'Top 10 Fatalities of States by Event for 2019',
                 height = 540)
fig.show()


In [306]:
#2018 fatalities by state 
dd_data= d_18.groupby(['STATE','EVENT_TYPE'], 
                       as_index=False).agg({'DEATHS_DIRECT': 'sum'})

states = list(dd_data['STATE'].unique()) 

fig = go.Figure()

for state in states:
    fig.add_trace(
                go.Bar(x=dd_data[dd_data['STATE'] == state].sort_values(
                       by= 'DEATHS_DIRECT', ascending = False)['EVENT_TYPE'].head(10), 
                       y= dd_data[dd_data['STATE'] == state].sort_values(
                       by= 'DEATHS_DIRECT', ascending = False)['DEATHS_DIRECT'].head(10), 
                       visible= False))

buttons= []

for i in range(len(states)):
    visible = [False]*len(states)
    if i == 0: 
        visible[i]= True
    else:
        visible[i] = True
        visible[i-1] = False
   
    buttons.append(dict(label = states[i],
                        method = 'update',           
                        args = [{'visible': visible}]))
 
fig.update_layout(updatemenus=list([
    dict(buttons = buttons)]))

fig.update_layout(title = 'Top 10 types of deaths by State (2018) ',
                 height = 540)
fig.show()


In [307]:
#2018 fatalities by state 
dd_data_19= d_19.groupby(['STATE','EVENT_TYPE'], 
                       as_index=False).agg({'DEATHS_DIRECT': 'sum'})

states = list(dd_data_19['STATE'].unique()) 

fig = go.Figure()

for state in states:
    fig.add_trace(
                go.Bar(x=dd_data_19[dd_data_19['STATE'] == state].sort_values(
                        by= 'DEATHS_DIRECT', ascending = False)['EVENT_TYPE'].head(10), 
                       y= dd_data_19[dd_data_19['STATE'] == state].sort_values(
                        by= 'DEATHS_DIRECT', ascending = False)['DEATHS_DIRECT'].head(10), 
                       visible= False))

buttons= []

for i in range(len(states)):
    visible = [False]*len(states)
    if i == 0: 
        visible[i]= True
    else:
        visible[i] = True
        visible[i-1] = False
   
    buttons.append(dict(label = states[i],
                        method = 'update',           
                        args = [{'visible': visible}]))
 
fig.update_layout(updatemenus=list([
    dict(buttons = buttons)]))

fig.update_layout(title = 'Top 10 types of deaths by State (2019) ',
                 height = 540)
fig.show()


In 2018, most of the deaths that were from storm events came from excessive heat, wildfire, heat, and rip currents. in 2019 it was tornadoes, heat, excessive heat, and flash floods. It appears that heat and excessive heat appear to be the biggest killers for both the years. For Delaware, there were no deaths caused by any storm events for 2018 and 2019. For Wyoming, in 2018 there was only two deaths caused by avalanches and in 2019 there were three deaths in which two were from heavy snow and one from an avalanche. As it appears that heat is the leading cause of death for individuals in regards to storm events. Wyoming and Delaware are not known to have cases of excessive heat. Also, these two states have very few deaths caused by these storm events which adds to them being safer than many other states such as Arizona who has over one hundred deaths by excessive heat alone each year. 

# When would be the most optimal time to move to one of these States?

Now that we have an understanding of how differnt types of storm events that happen during a year and in which states they are prevelent. It is important to gage when storm events take place for each selected state. Line graphs were constructed to see when the most optimal time would be to move to one of the selected states at the lowest occurance of storm events in a given year.

In [308]:
line_data= d_18.groupby(['STATE','MONTH_NAME'], as_index=False).agg({'EPISODE_ID': 'count'})

#calling dictionary function and passing key value pairs as arguments
month_map= dict(January = 1, February = 2, March = 3,
               April = 4, May = 5, June = 6, July =7,
               August = 8, September = 9, October = 10, 
               November = 11, December = 12)

#adding MONTH_NUM column to line_data dataframe
line_data['MONTH_NUM']= line_data['MONTH_NAME'].map(month_map)

events = list(line_data['STATE'].unique()) 

fig = go.Figure()

#adds the graph onto the figure does this by matching states with their respective event_type and Episdoe_ID
#boolean indexing each row to match it with with their events and corresponding MONTH_NAME and sorting the months by MONT_NUM
for event in events:
    fig.add_trace(
                go.Scatter(x=line_data[line_data['STATE'] == event].sort_values(
                       by = 'MONTH_NUM')['MONTH_NAME'], 
                       y= line_data[line_data['STATE'] == event].sort_values(by = 'MONTH_NUM')['EPISODE_ID'], 
                       mode = "lines+markers",
                       visible = False))

buttons= []

for i in range(len(events)):
    visible = [False]*len(events)
    if i == 0: 
        visible[i]= True
    else:
        visible[i] = True
        visible[i-1] = False
  
    buttons.append(dict(label = events[i],
                        method = 'update',           
                        args = [{'visible': visible}]))
  
fig.update_layout(updatemenus=list([
    dict(buttons = buttons)]))

fig.update_layout(title = 'Monthly Count of Storm Events by State (2018)',
                 height = 600)
fig.show()

In [309]:
line_data= d_19.groupby(['STATE','MONTH_NAME'], as_index=False).agg({'EPISODE_ID': 'count'})

month_map= dict(January = 1, February = 2, March = 3,
               April = 4, May = 5, June = 6, July =7,
               August = 8, September = 9, October = 10, 
               November = 11, December = 12)

line_data['MONTH_NUM']= line_data['MONTH_NAME'].map(month_map)

events = list(line_data['STATE'].unique()) 

fig = go.Figure()

for event in events:
    fig.add_trace(
                go.Scatter(x=line_data[line_data['STATE'] == event].sort_values(by = 'MONTH_NUM')['MONTH_NAME'], 
                       y= line_data[line_data['STATE'] == event].sort_values(by = 'MONTH_NUM')['EPISODE_ID'], 
                       mode = "lines+markers",
                       visible = False))

buttons= []

for i in range(len(events)):
    visible = [False]*len(events)
    if i == 0: 
        visible[i]= True
    else:
        visible[i] = True
        visible[i-1] = False
   
    buttons.append(dict(label = events[i],
                        method = 'update',           
                        args = [{'visible': visible}]))
   
fig.update_layout(updatemenus=list([
    dict(buttons = buttons)]))

fig.update_layout(title = 'Monthly Count of Storm Events by State (2019)',
                 height = 600)
fig.show()

In looking at the spread of storm events over time, Wyoming is more consistent than Delaware in predicting when the best time to move to these states. In 2018, Wyoming had over 250 weather events but this number gradually decreased until September where it only had five weather events. A similar trend could be seein in 2019 for Wyoming where there was a peak in July of 200 weather events, but it quickly decreased to fifty in September.

Delaware is a little different. The frequency of storm events are not consistent between the two years. For example, in 2018 there was a peak of 12 storm events and each subsequent month, the rates of storm events switch off between lows of 2-5 then up to 10-11). In 2019,  Delaware saw its peak in number of storm events in May and June with 25 and 26 events. However right before this in March it saw only 1 storm event and 13 in April. When examining the spread of storm events for 2018 and 2019, September seems like the most optimal time to move to Wyoming and late Winter to early Spring appears to be the most optimal time to move to Delaware. 

# Which counties should an individual move to in Delaware and  Wyoming? 

Finally, after studying which states had the lowest property damage, and understanding the different types of storm events that affect them, an analysis was conducted on the country level with choropleth maps to see which county an individual should move to by looking at the number of storm events within each country. 

In [310]:
#Delaware by county 2018
td_18_c= pd.DataFrame(d_18, columns = ['STATE','CZ_NAME','EVENT_TYPE', 'EPISODE_ID'])
tdd_18_c= td_18_c.groupby(["STATE",'CZ_NAME', "EVENT_TYPE"]).count().reset_index()
tdd_18_c.sort_values( by= 'EPISODE_ID', ascending = False, inplace = True)
tdd_18_c=tdd_18_c[tdd_18_c["STATE"]== 'DELAWARE']
de_bar= tdd_18_c.groupby(['CZ_NAME'], as_index=False).sum()

fig = go.Figure()
fig.add_trace( go.Bar(x= de_bar.sort_values(by ='EPISODE_ID')['CZ_NAME'].head(10), 
    y= de_bar.sort_values(by ='EPISODE_ID')['EPISODE_ID'].head(10)))

fig.update_layout(title = 'Delaware Counties with Least Storm Events (2018)',
                 height = 540)

In [311]:
#Delaware by county 2019
td_19_c= pd.DataFrame(d_19, columns = ['STATE','CZ_NAME','EVENT_TYPE', 'EPISODE_ID'])
tdd_19_c= td_19_c.groupby(["STATE",'CZ_NAME', "EVENT_TYPE"]).count().reset_index()
tdd_19_c.sort_values( by= 'EPISODE_ID', ascending = False, inplace = True)
tdd_19_c=tdd_19_c[tdd_19_c["STATE"]== 'DELAWARE']
de_bar_19= tdd_19_c.groupby(['CZ_NAME'], as_index=False).sum()

fig = go.Figure()
fig.add_trace( go.Bar(x= de_bar_19.sort_values(by ='EPISODE_ID')['CZ_NAME'].head(5), 
    y= de_bar_19.sort_values(by ='EPISODE_ID')['EPISODE_ID'].head(5)))

fig.update_layout(title = 'Delaware Counties with Least Storm Events (2019) ',
                 height = 540)

In [312]:
#Wyoming by county 2018
td_18_w= pd.DataFrame(d_18, columns = ['STATE','CZ_NAME','EVENT_TYPE', 'EPISODE_ID'])
tdd_18_w= td_18_w.groupby(["STATE",'CZ_NAME', "EVENT_TYPE"]).count().reset_index()
tdd_18_w.sort_values( by= 'EPISODE_ID', ascending = False, inplace = True)
tdd_18_w=tdd_18_w[tdd_18_w["STATE"]== 'WYOMING']
wy_bar= tdd_18_w.groupby(['CZ_NAME'], as_index=False).sum()

fig = go.Figure()
fig.add_trace( go.Bar(x= wy_bar.sort_values(by ='EPISODE_ID')['CZ_NAME'].head(15), 
    y= wy_bar.sort_values(by ='EPISODE_ID')['EPISODE_ID'].head(15)))

fig.update_layout(title = 'Wyoming Counties with Least Storm Events (2018) ',
                 height = 540)

In [313]:
#Wyoming by county 2019
td_19_w= pd.DataFrame(d_19, columns = ['STATE','CZ_NAME','EVENT_TYPE', 'EPISODE_ID'])
tdd_19_w= td_19_w.groupby(["STATE",'CZ_NAME', "EVENT_TYPE"]).count().reset_index()
tdd_19_w.sort_values( by= 'EPISODE_ID', ascending = False, inplace = True)
tdd_19_w=tdd_19_w[tdd_19_w["STATE"]== 'WYOMING']
wy_bar_19= tdd_19_w.groupby(['CZ_NAME'], as_index=False).sum()

fig = go.Figure()
fig.add_trace( go.Bar(x= wy_bar_19.sort_values(by ='EPISODE_ID')['CZ_NAME'].head(15), 
    y= wy_bar_19.sort_values(by ='EPISODE_ID')['EPISODE_ID'].head(15)))

fig.update_layout(title = 'Wyoming Counties with Least Storm Events (2019) ',
                 height = 540)

# Conclusion

The two safest counties to live in based on counts of weather events would be Delaware Beaches county, Delaware and Lincoln or Teton County, Wyoming. One should move to Delaware in late Winter-early Spring or to Wyoming in September if they want to limit their chances of encountering a weather event. Delaware Beaches only had 8 weather events in 2018 and 6 in 2019. Lincoln county only had one weather event in 2018 and two weather events in 2019. Teton county had one weather event in 2018 and two in 2019. In regards to the most weather events each state had by year, in 2018 Delaware had 19 flash floods and 14 costal floods. In 2019 Delware had 80 thunderstorms and 20 hail storms. For Wyoming in 2018 it had 564 cases of high winds, and 259 cases of hail storms. In 2019, Wyoming had 319 hail storms and 236 cases of high winds. Although the types of weather events change for each state per year, these states will not be dealing with cataqstrophic events such as tornados on a regular basis. heat and excessive heat take most peoples' lives each year.  
Although this analysis was a thorough investion into the finding the safest state and county in the US to live in, there could have been other approaches that would've yielded different responses. It would prove useful to investigate socio economical data such as literacy rates and poverty rates to find the safest county to live in while still having access to adequate healthcare. Finally, a more in depth analhysis would consider all the data available on storm events from 1950 to present which NOAA provides in their dataset. However, analyzing the  data from the past two years should give an sufficient estimate of the storm events in the US. 

link to Zenodo: [Jake's Zenodo]()

linik to github.io: [Jake's github.io](https://jcolt2997.github.io/DATS-6103-Individual-Project-2-Jake-Lieberfarb/)

link to github repository: [Jake's github page] ()